In [ ]:
import pandas as pd
import patsy
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
data = {
    "Block": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "Training Method (1)": [73, 76, 72, 74, 76, 75, 68, 72, 65, 62],
    "Training Method (2)": [81, 78, 80, 79, 71, 75, 72, 84, 73, 69],
    "Training Method (3)": [92, 89, 87, 90, 88, 86, 88, 87, 81, 78]
}

df = pd.DataFrame(data)

df_long = df.melt(id_vars='Block', 
                  var_name='Training', 
                  value_name='Score')

y = df_long["Score"].values.reshape(-1,1)
t = 3
b = 10

In [ ]:
X1 = np.kron(np.ones((b, 1)), np.identity(t))
X1

In [ ]:
X2 = np.kron(np.identity(b), np.ones((t, 1)))
X2

In [ ]:
X = np.hstack([X1, X2])
X

In [ ]:
y, X = patsy.dmatrices('Score ~ C(Block) + C(Training)', data=df_long)
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
beta_hat

In [ ]:
model_blocked = ols('Score ~ C(Block) + C(Training)', data=df_long).fit()
print(model_blocked.summary())

In [ ]:
anova_table = sm.stats.anova_lm(model_blocked, typ=2)
print(anova_table)

In [ ]:
mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
print("Estimated MSE:", mse)

In [ ]:
tukey = pairwise_tukeyhsd(endog=df_long['Score'],
                          groups=df_long['Training'],
                          alpha=0.10)
print(tukey.summary())

In [ ]:
MSE_blocked = model_blocked.mse_resid
model_CRD = ols('Score ~ C(Training)', data=df_long).fit()
MSE_CRD = model_CRD.mse_resid
reduction = (MSE_CRD - MSE_blocked) / MSE_CRD * 100

print("Completely Randomized Design MSE:", MSE_CRD)

print(f"Reduction in residual variance due to blocking: {reduction:.2f}%")